<a href="https://colab.research.google.com/github/yshnxd/solaris/blob/main/Welcome_To_Colab.py" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Libraries

In [2]:
# Install yfinance if not already installed
!pip install yfinance --upgrade --quiet

# Import core libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=54a3c2183b03be7d4675cef717d77765801bebdcb0d827d65c99f2bfff7ac026
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


# Collect Data

In [3]:
import yfinance as yf

ticker = 'AAPL'
start_date = '2024-07-15'
end_date = '2025-07-15'  # 1 year of 1 hr stock price data

data = yf.download(tickers=ticker, start=start_date, end=end_date, interval='1h')
print(data.head())

/tmp/ipython-input-3-2691904207.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=ticker, start=start_date, end=end_date, interval='1h')
[*********************100%***********************]  1 of 1 completed

Price                           Close        High         Low        Open  \
Ticker                           AAPL        AAPL        AAPL        AAPL   
Datetime                                                                    
2024-07-15 13:30:00+00:00  234.790100  237.229996  233.339996  236.531998   
2024-07-15 14:30:00+00:00  235.679993  236.310104  234.479996  234.820007   
2024-07-15 15:30:00+00:00  235.307999  235.949799  234.500000  235.570007   
2024-07-15 16:30:00+00:00  234.690002  235.880005  234.619995  235.320007   
2024-07-15 17:30:00+00:00  234.205002  234.800003  233.904007  234.659500   

Price                        Volume  
Ticker                         AAPL  
Datetime                             
2024-07-15 13:30:00+00:00  20535733  
2024-07-15 14:30:00+00:00   6741084  
2024-07-15 15:30:00+00:00   6661308  
2024-07-15 16:30:00+00:00   3851727  
2024-07-15 17:30:00+00:00   3774759  


# Preprocessing + Feature Creation

In [4]:
import numpy as np

# Calculate moving averages
data['MA_10'] = data['Close'].rolling(window=10).mean()
data['MA_50'] = data['Close'].rolling(window=50).mean()

# Calculate Relative Strength Index (RSI)
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['RSI'] = calculate_rsi(data)

# Calculate Daily Returns
data['Daily Return'] = data['Close'].pct_change()

# Drop NaN values
data = data.dropna()

# Define features and target
features = ['Close', 'MA_10', 'MA_50', 'RSI', 'Daily Return']
X = data[features].values
y = (data['Close'].shift(-1) > data['Close']).astype(int).values  # 1 if price goes up, 0 if down
X = X[:-1]  # Align X with y
y = y[:-1]  # Align y with X


# Reshape Data for LSTM Input

In [5]:
# Function to create sequences for LSTM
def create_sequences(data, target, time_steps=1):
    X_seq, y_seq = [], []
    for i in range(len(data) - time_steps):
        X_seq.append(data[i:(i + time_steps)])
        y_seq.append(target[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

# Set time steps for LSTM
time_steps = 10
X_seq, y_seq = create_sequences(X, y, time_steps)

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)


# Build and Train the LSTM Model

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Initialize the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.5445 - loss: 0.6948 - val_accuracy: 0.4583 - val_loss: 0.6996
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5130 - loss: 0.6982 - val_accuracy: 0.4583 - val_loss: 0.7055
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5085 - loss: 0.6938 - val_accuracy: 0.4583 - val_loss: 0.7079
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5081 - loss: 0.6957 - val_accuracy: 0.4583 - val_loss: 0.6951
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5415 - loss: 0.6900 - val_accuracy: 0.4583 - val_loss: 0.7081
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5250 - loss: 0.6958 - val_accuracy: 0.4583 - val_loss: 0.7039
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5312 - loss: 0.6934 - val_accuracy: 0.4583 - val_loss: 0.6985
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5252 - loss: 0.6925 - val_accuracy: 0.4583 - val_loss: 0.

# Evaluate The Model

In [10]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4364 - loss: 0.7030 
Accuracy: 0.46


# Make Predictions and Implement Trading Strategy

In [11]:
# Make predictions
predictions = model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)

# Implement trading strategy
initial_balance = 10000  # Starting balance in USD
balance = initial_balance
position = 0  # Number of shares

for i in range(len(predicted_classes)):
    current_price = X_test[i][-1][0]  # Get the last price in the sequence
    predicted_movement = predicted_classes[i][0]

    if predicted_movement == 1 and balance >= current_price:
        # Buy stock
        shares_to_buy = int(balance // current_price)
        if shares_to_buy > 0:
            position += shares_to_buy
            balance -= shares_to_buy * current_price
            print(f"Buying {shares_to_buy} shares at {current_price:.2f}")

    elif predicted_movement == 0 and position > 0:
        # Sell stock
        balance += position * current_price
        print(f"Selling {position} shares at {current_price:.2f}")
        position = 0

# Calculate final balance including the value of the remaining shares
final_balance = balance + (position * X_test[-1][-1][0])
profit = final_balance - initial_balance
print(f"Final balance: ${final_balance:.2f}")
print(f"Profit: ${profit:.2f}")


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step
Buying 50 shares at 199.69
Final balance: $9922.25
Profit: $-77.75
